<a href="https://colab.research.google.com/github/eruu-data29/project1/blob/main/finance_dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import openai
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from transformers import pipeline
from prophet import Prophet
import plotly.graph_objects as go

# Set OpenAI API Key (Replace with your own)
openai.api_key = st.secrets["OPENAI_API_KEY"]
# Load Transformer Model for Summarization
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Streamlit Dashboard UI
st.set_page_config(page_title="📈 AI-Powered Market Insights", layout="wide")
st.title("📈 AI-Powered Financial News & Market Forecast")
st.markdown("Enter financial news below to get AI-generated insights and trend forecasts.")

# User Input
news_text = st.text_area("📰 Enter Financial News Article or Speech", height=200)

# Analysis Options
analysis_type = st.radio("Select Analysis Type:", ["Market Insight", "Sentiment Analysis", "Trend Forecasting", "All"])

# Advanced Filters
st.sidebar.header("🔍 Advanced Settings")
summary_length = st.sidebar.slider("Summary Length (words)", 30, 200, 100)
forecast_days = st.sidebar.slider("Forecast Days", 3, 30, 7)

# Dummy historical data for trend forecasting
date_rng = pd.date_range(start="2024-01-01", end=datetime.today(), freq='D')
historical_data = pd.DataFrame({
    "ds": date_rng,
    "y": np.cumsum(np.random.randn(len(date_rng))) + 50  # Simulated stock price movements
})

# AI Processing
if st.button("🔍 Analyze"):
    if news_text:
        with st.spinner("Generating AI insights..."):
            # Summarization
            summary = summarizer(news_text, max_length=summary_length, min_length=30, do_sample=False)[0]["summary_text"]

            # AI Financial Market Insight from GPT-4
            response = openai.ChatCompletion.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a financial market analyst."},
                    {"role": "user", "content": f"Analyze this financial news and provide a market insight: {news_text}"}
                ]
            )
            insight = response["choices"][0]["message"]["content"]

            # Display Insights
            if analysis_type in ["Market Insight", "All"]:
                st.subheader("📝 AI-Generated Summary")
                st.write(summary)

                st.subheader("📊 AI Market Insight")
                st.write(insight)

            # Sentiment Analysis (Using GPT response tokens as a mock sentiment score)
            sentiment_score = response["usage"]["total_tokens"] / 1000  # Mock Sentiment Score
            sentiment_label = "Bullish" if sentiment_score < 0.5 else "Bearish"

            if analysis_type in ["Sentiment Analysis", "All"]:
                st.subheader("📈 Sentiment Analysis")
                st.write(f"**Sentiment:** {sentiment_label} (Score: {sentiment_score:.2f})")
                if sentiment_score >= 0.5:
                    st.warning("⚠️ High bearish sentiment detected! Market may react negatively.")

            # Trend Forecasting using Prophet
            if analysis_type in ["Trend Forecasting", "All"]:
                st.subheader("📈 Market Trend Forecasting")

                # Train Prophet Model
                model = Prophet()
                model.fit(historical_data)

                # Create Future Dataframe
                future = model.make_future_dataframe(periods=forecast_days)
                forecast = model.predict(future)

                # Plot Forecast
                fig = go.Figure()
                fig.add_trace(go.Scatter(x=forecast["ds"], y=forecast["yhat"], mode="lines", name="Predicted Trend"))
                fig.add_trace(go.Scatter(x=historical_data["ds"], y=historical_data["y"], mode="lines", name="Historical Data"))
                st.plotly_chart(fig)

                # Show Forecast Data
                st.dataframe(forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]].tail(forecast_days))

    else:
        st.error("❌ Please enter financial news.")